In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import pathlib

from tqdm import tqdm

import re

import usaddress

from difflib import get_close_matches as clmatch

import geopandas as gpd

from shapely import Point

import math

import logging

path = pathlib.Path().resolve()

data_path = path.parent / "Dropbox" / "2019 MV Data by Town" / "Vehicles_2022" / "Compiled"

raw_data = pd.read_csv(data_path / "2019-21_data_compiled_RN_100323.csv", chunksize = 1000)

In [113]:
# Get first 100,000 rows
raw_data_100 = pd.DataFrame([])
i = 0
for chunk in tqdm(raw_data):
    if i < 100:
        raw_data_100 = pd.concat([raw_data_100, chunk])
        i+=1
    else:
        break

100it [00:01, 58.22it/s]


In [90]:
usaddress.parse("cdjksf")

[('cdjksf', 'CountryName')]

In [78]:
def valid_zip(zip_code):
    zip_str = str(zip_code)
    split_zip = re.split("-", zip_str)
    if len(split_zip) == 2:
        if valid_zip(split_zip[0]) & valid_zip(split_zip[1]):
            return 2
        elif valid_zip(split_zip[0]):
            return 3
        else:
            return 0
    elif len(split_zip) == 1:
        # MUST ADDRESS STARTING "Os"
        # starting_o = re.match('^O', split_zip[0])
        matched = re.match("^\s*[0-9]*[0-9]{4}\.?0?\s*$", split_zip[0])
        if matched:
            return 1
        else:
            return 0
    else:
        return 0

In [51]:
# Test valid zip
raw_data = pd.read_csv(data_path / "2019-21_data_compiled_RN_100323.csv", chunksize = 1000)
raw_zips_all = pd.DataFrame([])
i = 0
for chunk in raw_data:
    if i % 100 == 0:
        print(f"On chunk {i})")
    zips = chunk["zip"]
    raw_zips_all = pd.concat([raw_zips_all, zips])
    i +=1

On chunk 0)
On chunk 100)
On chunk 200)
On chunk 300)
On chunk 400)
On chunk 500)
On chunk 600)
On chunk 700)
On chunk 800)
On chunk 900)
On chunk 1000)
On chunk 1100)
On chunk 1200)
On chunk 1300)
On chunk 1400)
On chunk 1500)
On chunk 1600)
On chunk 1700)
On chunk 1800)
On chunk 1900)
On chunk 2000)
On chunk 2100)
On chunk 2200)
On chunk 2300)
On chunk 2400)
On chunk 2500)
On chunk 2600)
On chunk 2700)
On chunk 2800)
On chunk 2900)
On chunk 3000)
On chunk 3100)
On chunk 3200)
On chunk 3300)
On chunk 3400)
On chunk 3500)
On chunk 3600)
On chunk 3700)
On chunk 3800)
On chunk 3900)
On chunk 4000)
On chunk 4100)
On chunk 4200)
On chunk 4300)
On chunk 4400)
On chunk 4500)
On chunk 4600)
On chunk 4700)
On chunk 4800)
On chunk 4900)
On chunk 5000)
On chunk 5100)
On chunk 5200)
On chunk 5300)
On chunk 5400)
On chunk 5500)
On chunk 5600)
On chunk 5700)


In [52]:
raw_zips_all = raw_zips_all.rename(columns = {0: "zip"})

In [54]:
raw_zips_all.to_csv(path / "raw_zips.csv")

In [28]:
raw_zips_all = pd.read_csv(path / "raw_zips.csv")

C:\Users\rajan\AppData\Local\Temp\ipykernel_14904\1286788364.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_zips_all = pd.read_csv(path / "raw_zips.csv")


In [66]:
valid_zips = raw_zips_all["zip"].apply(lambda x: valid_zip(x))

In [67]:
valid_zips_df = pd.DataFrame(list(valid_zips), columns = ["valid_zips"])

In [70]:
raw_zips_all[valid_zips_df["valid_zips"]==0]["zip"].unique()

array(['     ', '0', '820', '659', '830', '608', '963', '610', '607',
       '705', nan, '968', '000V6', ' ', 'CT', 'LN', '12', '5D', 'B',
       '134', 'D', 'D2', '000L4', '646', 'STH', '801', '604', '16', '616',
       '718', '773', '8', '259', '603', '802', '000CT', '924', '000M4',
       'X6040', '959', '608.0', '831', '651', '602', '777', '692', '678',
       'X6751', '0.0', '487', '85', '307', '90.0', '71.0', '23.0', '6.0',
       '000T4', '927', '754', '717', 'M1G2J', '628', '623', '622', '674',
       '636', 'T5S1K     ', '          ', '458', '681', '601', '669',
       '690', '633.0', '685', '646.0', '719', '666', '784', '917',
       '765.0', '707', '605', 735, 727, '926', '907', '-', '460', '689',
       '682', 'O6472', 'O6471', 'O6762', 'O6517', 'O6443', 'O6473',
       'O6256', 'O6437', 'O6460', 'O6405', 'O8054', 'O6511', 'O6340',
       'O6513', 'O6067', 'O6442', 'O6518', 'O6357', 'O6516', 'O6525',
       'O6415', 'O6492', 'O6512', 'O6477', 'O6413', 'O6051', 'O6708',
    

# VINs

In [92]:
# Test valid zip
raw_data = pd.read_csv(data_path / "2019-21_data_compiled_RN_100323.csv", chunksize = 1000)
raw_vins_all = pd.DataFrame([])
i = 0
for chunk in raw_data:
    if i % 100 == 0:
        print(f"On chunk {i})")
    vins = chunk["vehicle_id"]
    raw_vins_all = pd.concat([raw_vins_all, vins])
    i +=1

On chunk 0)
On chunk 100)
On chunk 200)
On chunk 300)
On chunk 400)
On chunk 500)
On chunk 600)
On chunk 700)
On chunk 800)
On chunk 900)
On chunk 1000)
On chunk 1100)
On chunk 1200)
On chunk 1300)
On chunk 1400)
On chunk 1500)
On chunk 1600)
On chunk 1700)
On chunk 1800)
On chunk 1900)
On chunk 2000)
On chunk 2100)
On chunk 2200)
On chunk 2300)
On chunk 2400)
On chunk 2500)
On chunk 2600)
On chunk 2700)
On chunk 2800)
On chunk 2900)
On chunk 3000)
On chunk 3100)
On chunk 3200)
On chunk 3300)
On chunk 3400)
On chunk 3500)
On chunk 3600)
On chunk 3700)
On chunk 3800)
On chunk 3900)
On chunk 4000)
On chunk 4100)
On chunk 4200)
On chunk 4300)
On chunk 4400)
On chunk 4500)
On chunk 4600)
On chunk 4700)
On chunk 4800)
On chunk 4900)
On chunk 5000)
On chunk 5100)
On chunk 5200)
On chunk 5300)
On chunk 5400)
On chunk 5500)
On chunk 5600)
On chunk 5700)


In [95]:
raw_vins_all = raw_vins_all.rename(columns = {0: "raw_vins"})

In [96]:
raw_vins_all.to_csv(path / "raw_vins.csv")

In [100]:
raw_vins_all["vin_length"]=raw_vins_all["raw_vins"].str.len()

In [110]:
raw_vins_all[raw_vins_all["vin_length"]<7]

,raw_vins,vin_length
17,549285,6.0
260,1747,4.0
288,41518,5.0
593,8939,4.0
944,37014,5.0
...,...,...
5769473,111129,6.0
5769476,131799,6.0
5769671,1198,4.0
5769739,11725,5.0


In [114]:
raw_data_100["vin_length"] = raw_data_100["vehicle_id"].str.len()

In [118]:
raw_data_100[raw_data_100["vin_length"]<9][["vehicle_make", "vehicle_model", "vehicle_id"]]

,vehicle_make,vehicle_model,vehicle_id
17,HIGHL,TRAILER,549285
70,BUICK,COUPE 46,33441303
260,HALE,BT185,1747
288,SHIPM,11201,41518
593,STRAT,24 FT,8939
...,...,...,...
36870,Ford,3 WNDWCO,18179022
36963,NORTO,ES2,84836
37766,SHORE,PDL35,901224
38250,VW,BEETLE,1450866


## Get makes

In [233]:
raw_data = pd.read_csv(data_path / "2019-21_data_compiled_RN_100323.csv", chunksize = 1000)

In [234]:
# Get all makes
raw_makes_counts = pd.DataFrame([])
i = 0
for chunk in tqdm(raw_data):
    makes_counts = chunk.groupby("vehicle_make").count()
    raw_makes_counts = pd.concat([raw_makes_counts, makes_counts])

5788it [03:32, 27.20it/s]


In [236]:
raw_makes_counts = raw_makes_counts[["Unnamed: 0"]].rename(columns = {"Unnamed: 0": "Count"})
raw_makes_counts = raw_makes_counts.groupby("vehicle_make").sum()
raw_makes_counts = raw_makes_counts.reset_index()
raw_makes_counts = raw_makes_counts.sort_values("Count", ascending = False)
raw_makes_counts.to_csv(path / "raw_makes_counts.csv")

In [268]:
raw_makes_counts = pd.read_csv(path / "raw_makes_counts.csv")

In [273]:
raw_makes_counts["vehicle_make"] = raw_makes_counts["vehicle_make"].str.strip()

In [276]:
raw_makes_counts_grouped = raw_makes_counts.groupby("vehicle_make").sum().reset_index()

In [301]:
raw_makes_counts_grouped.sort_values("Count", ascending = False).head(30)

,vehicle_make,Unnamed: 0,Count,vehicle_make_matched,vehicle_make_matched_90,vehicle_make_matched_80,vehicle_make_matched_70,vehicle_make_matched_60,vehicle_make_matched_50
4398,HONDA,15546,586909,HONDA,HONDA,HONDA,HONDA,HONDA,HONDA
9473,TOYOT,22351,562592,TOYOTA,TOYOTA,TOYOTA,TOYOTA,TOYOTA,TOYOTA
3469,FORD,12255,521515,FORD,FORD,FORD,FORD,FORD,FORD
1818,CHEVR,6486,435979,CHEVROLET,,,CHEVROLET,CHEVROLET,CHEVROLET
6746,NISSA,7959,353145,NISSAN,NISSAN,NISSAN,NISSAN,NISSAN,NISSAN
8859,SUBAR,31344,326739,SUBARU,SUBARU,SUBARU,SUBARU,SUBARU,SUBARU
4880,JEEP,17247,257056,JEEP,JEEP,JEEP,JEEP,JEEP,JEEP
4558,HYUND,5366,188301,HYUNDAI,,HYUNDAI,HYUNDAI,HYUNDAI,HYUNDAI
1059,BMW,5046,149478,BMW,BMW,BMW,BMW,BMW,BMW
2682,DODGE,6349,129082,DODGE,DODGE,DODGE,DODGE,DODGE,DODGE


In [300]:
raw_makes_counts_grouped["vehicle_make_matched_90"] = raw_makes_counts_grouped["vehicle_make"].apply(lambda x: clmatch_plus(x.upper(),real_makes, n = 6, cutoff = 0.90))
raw_makes_counts_grouped["vehicle_make_matched_80"] = raw_makes_counts_grouped["vehicle_make"].apply(lambda x: clmatch_plus(x.upper(),real_makes, n = 6, cutoff = 0.80))
raw_makes_counts_grouped["vehicle_make_matched_70"] = raw_makes_counts_grouped["vehicle_make"].apply(lambda x: clmatch_plus(x.upper(),real_makes, n = 6, cutoff = 0.70))
raw_makes_counts_grouped["vehicle_make_matched_60"] = raw_makes_counts_grouped["vehicle_make"].apply(lambda x: clmatch_plus(x.upper(),real_makes, n = 6, cutoff = 0.60))
raw_makes_counts_grouped["vehicle_make_matched_50"] = raw_makes_counts_grouped["vehicle_make"].apply(lambda x: clmatch_plus(x.upper(),real_makes, n = 6, cutoff = 0.50))

In [302]:
raw_makes_counts_grouped.to_csv(path / "raw_makes_counts_grouped.csv")

In [206]:
def clmatch_plus(to_match, lst, n, cutoff):
    matches = clmatch(to_match, lst, n, cutoff)
    if len(matches) > 0:
        return matches[0]
    else:
        return ""

In [196]:
real_makes = list(pd.read_csv(path / "real_makes.csv")["real_makes"])

In [335]:
raw_data_1000[raw_data_1000["vehicle_make"]=="PIAGG"]

,Unnamed: 0,record_from,name,street,city,state,zip,vehicle_year,vehicle_make,vehicle_model,vehicle_class,vehicle_id,lease_street,UID,lease_city,lease_state,lease_zip
5176,5176,002_Ansonia_MV_21.xlsx,BIASUCCI NATALIE A,57 ROCKWOOD AVE,ANSONIA,CT,6401,2013.0,PIAGG,BV350IE,12.0,ZAPM690T8D5001807,NaN,NaN,NaN,NaN,NaN
41663,41663,005_Barkhamsted_MV_21.xlsx,NIEHAUS CARLA A,49 ROBIN DR,BARKHAMSTED,CT,6063,2009.0,PIAGG,FLY 50,12.0,ZAPM578F994001057,NaN,NaN,NaN,NaN,NaN
43381,43381,006_Beacon_Falls_MV_21.xlsx,BELLI GERALD J JR,25 STODDARD PL,BEACON FALLS,CT,6403,2020.0,PIAGG,LIBERTY,12.0,ZAPM898G4L7001503,NaN,NaN,NaN,NaN,NaN
47630,47630,006_Beacon_Falls_MV_21.xlsx,SALERNO DAVID C,242 SKOKORAT RD,BEACON FALLS,CT,6403,2021.0,PIAGG,BV350IE,12.0,ZAPMA20S9M5000940,NaN,NaN,NaN,NaN,NaN
51583,51583,007_Berlin_MV_21.xlsx,CALAFIORE SALVATORE J,151 WARNER RD,BERLIN,CT,6037,2006.0,PIAGG,X9 EVOLU,12.0,ZAPM270W365001503,NaN,NaN,NaN,NaN,NaN
55906,55906,007_Berlin_MV_21.xlsx,EGAZARIAN EDWARD C,1183 WORTHINGTON RDG,BERLIN,CT,6037,2009.0,PIAGG,BV500,12.0,ZAPM340W895301019,NaN,NaN,NaN,NaN,NaN
56931,56931,007_Berlin_MV_21.xlsx,FRIGERI JOHN U,PO BOX 7134,KENSINGTON,CT,6037,2005.0,PIAGG,BV200,12.0,ZAPM289L255002333,NaN,NaN,NaN,NaN,NaN
62231,62231,007_Berlin_MV_21.xlsx,MALS RONALD C,99 YORK RD,KENSINGTON,CT,6037,2012.0,PIAGG,FLY 50,12.0,ZAPM578F2C4000677,NaN,NaN,NaN,NaN,NaN
145553,145553,014_Branford_MV_21.xlsx,MUNROE CHRISTOPHER,2 BRIDGE ST APT 1,BRANFORD,CT,6405,2008.0,PIAGG,FLY 50,12.0,ZAPM578F684000222,,67124.0,,,
173958,173958,015_Bridgeport_MV_21.xlsx,DELVALLE CARLOS C,264 UNION AVE APT 312,BRIDGEPORT,CT,6607,2007.0,PIAGG,BV250,12,ZAPM289LX75102036,NaN,NaN,NaN,NaN,NaN


# Addresses

In [343]:
import geopandas

In [375]:
# Get all raw addresses
raw_data = pd.read_csv(data_path / "2019-21_data_compiled_RN_100323.csv", chunksize = 1000)
i = 0
raw_addresses = pd.DataFrame([])

for chunk in raw_data:
    if i % 100 == 0:
        print(f"On chunk {i}")
        
    address_data = chunk[["street", "city", "state", "zip"]]
    raw_addresses = pd.concat([raw_addresses, address_data])
    
    i += 1

On chunk 0
On chunk 100
On chunk 200
On chunk 300
On chunk 400
On chunk 500
On chunk 600
On chunk 700
On chunk 800
On chunk 900
On chunk 1000
On chunk 1100
On chunk 1200
On chunk 1300
On chunk 1400
On chunk 1500
On chunk 1600
On chunk 1700
On chunk 1800
On chunk 1900
On chunk 2000
On chunk 2100
On chunk 2200
On chunk 2300
On chunk 2400
On chunk 2500
On chunk 2600
On chunk 2700
On chunk 2800
On chunk 2900
On chunk 3000
On chunk 3100
On chunk 3200
On chunk 3300
On chunk 3400
On chunk 3500
On chunk 3600
On chunk 3700
On chunk 3800
On chunk 3900
On chunk 4000
On chunk 4100
On chunk 4200
On chunk 4300
On chunk 4400
On chunk 4500
On chunk 4600
On chunk 4700
On chunk 4800
On chunk 4900
On chunk 5000
On chunk 5100
On chunk 5200
On chunk 5300
On chunk 5400
On chunk 5500
On chunk 5600
On chunk 5700


In [379]:
raw_addresses["address_joined"] = raw_addresses["street"] + ", " + raw_addresses["city"] + ", " + raw_addresses["state"]

In [382]:
raw_addresses.to_csv(path / "raw_addresses.csv")

In [423]:
addresses_geocoded = gpd.GeoDataFrame([], columns = ["geometry", "address"], geometry = "geometry")

In [424]:
for i in tqdm(range(200)):
    geom, address = standardize_address(raw_addresses["address_joined"][i])
    addresses_geocoded.loc[i] = [geom, address]
    
    if (i % 50 == 0) & (i>0):
        print(f"Saving gpkg file, on index{i}")
        addresses_geocoded.set_geometry(col='geometry', inplace=True)
        addresses_geocoded.to_file(path / "geocoded_addresses.gpkg") 

 26%|██▌       | 51/200 [00:36<02:30,  1.01s/it]

Saving gpkg file, on index{i}


 50%|█████     | 101/200 [01:13<01:22,  1.19it/s]

Saving gpkg file, on index{i}


 75%|███████▌  | 150/200 [01:49<00:32,  1.53it/s]

Saving gpkg file, on index{i}


100%|██████████| 200/200 [02:28<00:00,  1.35it/s]


In [425]:
# Check
merged_geocoded = raw_addresses[0:200].join(addresses_geocoded)

In [430]:
pd.set_option('display.max_colwidth', None)

In [438]:
standardize_address("2 SHADBLOW LANE , ANDOVER , CT")

[<POINT (-72.374 41.754)>,
 '2, Shadblow Lane, Andover, Capitol Planning Region, Connecticut, 06232, United States']

In [432]:
merged_geocoded[30:70]

,street,city,state,zip,address_joined,geometry,address
30,11 CENTER ST,ANDOVER,CT,6232,"11 CENTER ST , ANDOVER , CT",POINT (-72.37157 41.73743),"11, Center Street, Andover, Capitol Planning Region, Connecticut, 06232, United States"
31,1 TIMES FARM RD,ANDOVER,CT,6232,"1 TIMES FARM RD , ANDOVER , CT",POINT (-72.40041 41.75987),"1, Times Farm Road, Andover, Capitol Planning Region, Connecticut, 06232, United States"
32,2 SHADBLOW LN,ANDOVER,CT,6232,"2 SHADBLOW LN , ANDOVER , CT",POINT (-72.37443 41.75447),"2, Shadblow Lane, Andover, Capitol Planning Region, Connecticut, 06232, United States"
33,43 JUROVATY RD,ANDOVER,CT,6232,"43 JUROVATY RD , ANDOVER , CT",POINT (-72.39221 41.72211),"43, Jurovaty Road, Andover, Capitol Planning Region, Connecticut, 06232, United States"
34,43 JUROVATY RD,ANDOVER,CT,6232,"43 JUROVATY RD , ANDOVER , CT",POINT (-72.39221 41.72211),"43, Jurovaty Road, Andover, Capitol Planning Region, Connecticut, 06232, United States"
35,43 JUROVATY RD,ANDOVER,CT,6232,"43 JUROVATY RD , ANDOVER , CT",POINT (-72.39221 41.72211),"43, Jurovaty Road, Andover, Capitol Planning Region, Connecticut, 06232, United States"
36,PO BOX 335,ANDOVER,CT,6232,"PO BOX 335 , ANDOVER , CT",POINT EMPTY,False
37,PO BOX 335,ANDOVER,CT,6232,"PO BOX 335 , ANDOVER , CT",POINT EMPTY,False
38,PO BOX 335,ANDOVER,CT,6232,"PO BOX 335 , ANDOVER , CT",POINT EMPTY,False
39,57 SHODDY MILL RD,ANDOVER,CT,6232,"57 SHODDY MILL RD , ANDOVER , CT",POINT (-72.38809 41.73956),"57, Shoddy Mill Road, Andover, Capitol Planning Region, Connecticut, 06232, United States"


In [420]:
def standardize_address(address):
    geocoded = geopandas.tools.geocode(
    address,
    provider="nominatim",
    user_agent="Yale",
    timeout=10)
    geometry = geocoded["geometry"][0]
    standardized = geocoded["address"][0]
    if standardized:
        return [geometry, standardized]
    else:
        return [Point(), False]

* Might use Google's geo-coding API to geo-code these as well...

# Match VINs

In [3]:
# Attempt match on first 1000k
raw_data_1000.to_csv(path / "raw_data_1000.csv")

NameError: name 'raw_data_1000' is not defined

In [466]:
raw_data_1000_merge = raw_data_1000[["vehicle_make", "vehicle_model", "vehicle_id"]]

In [485]:
raw_data_1000_merge["VIN_MERGE"] = raw_data_1000["vehicle_id"].str[0:8]+"*"+raw_data_1000["vehicle_id"].str[9:11]

C:\Users\rajan\AppData\Local\Temp\ipykernel_14904\121427070.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data_1000_merge["VIN_MERGE"] = raw_data_1000["vehicle_id"].str[0:8]+"*"+raw_data_1000["vehicle_id"].str[9:11]


In [486]:
vin_merge = raw_data_1000_merge.merge(nhtsa_full,
                                    left_on = "VIN_MERGE",
                                    right_on = "VIN",
                                    how = "left")

In [494]:
unmatched_vins = vin_merge[vin_merge["VIN"].isna()]["VIN_MERGE"].unique()

In [530]:
all_vins_1000 = raw_data_1000_merge["VIN_MERGE"].unique()

In [534]:
# Create a blank DataFrame to fill with queries
colnames = ["EVDriveUnit", "FuelTypePrimary", "ElectrificationLevel", "VIN"]

resp_df_out = pd.DataFrame([], columns = colnames)

# Proceed through unmatched VINs one by one and query the NHTSA database
for vin in tqdm(all_vins_1000[0:10000]):
    # Check length
    if len(vin.strip()) == 11:
        # Set URL
        url = (f"https://vpic.nhtsa.dot.gov/api/vehicles/DecodeVin/{vin.strip()}?format=csv")

        # Download response
        resp_df = pd.read_csv(url)

        # Create a DataFrame for this response
        EVDriveUnit = resp_df.iloc[resp_df[resp_df["variable"]=="EV Drive Unit"].index,3].tolist()[0]
        FuelTypePrimary = resp_df.iloc[resp_df[resp_df["variable"]=="Fuel Type - Primary"].index,3].tolist()[0]
        ElectrificationLevel = resp_df.iloc[resp_df[resp_df["variable"]=="Electrification Level"].index, 3].tolist()[0]

    else:
        EVDriveUnit = np.NaN
        FuelTypePrimary = np.NaN
        ElectrificationLevel = np.NaN
    
    VIN = vin
    resp_df_add = pd.DataFrame([[EVDriveUnit, FuelTypePrimary, ElectrificationLevel, VIN]], columns = colnames)

    # Add this to the master output DataFrame
    resp_df_out = pd.concat([resp_df_out, resp_df_add])

 21%|██        | 2093/10000 [09:39<36:28,  3.61it/s]  


HTTPError: HTTP Error 404: Not Found

In [537]:
url

'https://vpic.nhtsa.dot.gov/api/vehicles/DecodeVin/1.36379E*12?format=csv'

In [535]:
resp_df_out[resp_df_out["FuelTypePrimary"]=="Electric"]

,EVDriveUnit,FuelTypePrimary,ElectrificationLevel,VIN
0,NaN,Electric,PHEV (Plug-in Hybrid Electric Vehicle),1G1RC6S5*GU
0,NaN,Electric,PHEV (Plug-in Hybrid Electric Vehicle),JHMZC5F3*MC
0,NaN,Electric,BEV (Battery Electric Vehicle),5YJYGDEE*MF
0,NaN,Electric,PHEV (Plug-in Hybrid Electric Vehicle),JTDKARFP*L3


In [564]:
raw_data_1000["vehicle_id"].str.strip().str.isalnum()

0         True
1         True
2         True
3         True
4         True
          ... 
999995    True
999996    True
999997    True
999998    True
999999    True
Name: vehicle_id, Length: 1000000, dtype: object

In [569]:
vins_2 =raw_data_1000[["vehicle_id"]]

# Full processing

## Prepare data and import matching sets

In [708]:
# raw_data_1000.to_csv(path / "raw_data_1000.csv")

In [4]:
raw_data_1000 = pd.read_csv(path / "data" / "raw_data_1000.csv")

NameError: name 'path' is not defined

In [9]:
# Load NHTSA data
nhtsa_cleaned = pd.read_csv(path / "ignored-data" / "NHTSA_cleaned.csv")

# Simplify the cleaned file
nhtsa_cleaned_simple = nhtsa_cleaned[["VIN", "Manufacturer", "Model", "ModelYear", "FuelTypePrimary", "ElectrificationLevel"]]

C:\Users\rajan\AppData\Local\Temp\ipykernel_7232\1728378471.py:2: DtypeWarning: Columns (7,9,10,13,20,25,30,34,42,44,56,58,68,78,80,114,115,138,146) have mixed types. Specify dtype option on import or set low_memory=False.
  nhtsa_cleaned = pd.read_csv(path / "ignored-data" / "NHTSA_cleaned.csv")


In [10]:
raw_data_1000 = raw_data_1000.loc[:, ~raw_data_1000.columns.isin(["Unnamed: 0.1", "Unnamed: 0"])]

## Required functions

In [90]:
valid = valid_vins(raw_data_1000[["vehicle_id"]][0:50])

In [97]:
matched = match_vins(valid)

A total of 27 VINs were not matched


 26%|██▌       | 7/27 [00:01<00:05,  3.80it/s]

'float' object has no attribute 'strip'


100%|██████████| 27/27 [00:06<00:00,  3.94it/s]


This number of unmatched VINs was reduced by 24
Remaining unmatched VINs is 3


In [96]:
def match_vins(valid_vin_list):
    """Take in a df of valid vins (that could be a valid vin or NA) and match it"""
    match = valid_vin_list.merge(nhtsa_cleaned_simple,
                                 left_on = "vin_corrected",
                                 right_on = "VIN",
                                 how = "left")
    # Get unique unmatched vins
    unmatched_vins = list(match[match["VIN"].isna()]["vin_corrected"].unique())
    
    # Print how many
    print(f"A total of {len(unmatched_vins)} VINs were not matched")
    
    # Variables to download
    variables = ["Manufacturer Name", "Model", "Model Year", "Fuel Type - Primary", "Electrification Level"]
    variables_shortnames = ["Manufacturer", "Model", "ModelYear", "FuelTypePrimary", "ElectrificationLevel"]
    
    # Go get them
    for vin in tqdm(unmatched_vins):
        try:
            
            
            url = (f"https://vpic.nhtsa.dot.gov/api/vehicles/DecodeVin/{vin.strip()}?format=csv")

            # Download response
            resp_df = pd.read_csv(url)
            
            # Extract needed
            resp_df = resp_df.loc[resp_df["variable"].isin(variables), ["variable", "value"]].T
            resp_df.columns = resp_df.iloc[0]
            resp_df = resp_df.drop("variable", axis = 0)
            valid_response = not(resp_df["Fuel Type - Primary"].isna()[0])
            
            # Add back to the DF
            if valid_response:
                for i in range(len(variables)):
                    match.loc[match[match["vin_corrected"]==vin].index, variables_shortnames[i]] = resp_df[variables[i]][0]
                match.loc[match[match["vin_corrected"]==vin].index, "VIN"] = vin
            else:
                for i in range(len(variables)):
                    match.loc[match[match["vin_corrected"]==vin].index, variables_shortnames[i]] = np.NaN
        except BaseException as e:
            print(e)
            pass
            
    remaining_unmatched = list(match[match["VIN"].isna()]["vin_corrected"].unique())
    
    print(f"This number of unmatched VINs was reduced by {len(unmatched_vins) - len(remaining_unmatched)}")
    print(f"Remaining unmatched VINs is {len(remaining_unmatched)}")
    
    return match

In [11]:
def valid_vins(vin_list):
    """Take in a DF of VINs and determine when they are possibly valid"""
    # Check all alphanumerics
    vin_list["vin_alnum_check"] = vin_list["vehicle_id"].str.strip().str.isalnum()
    vin_list["vin_len_check"] = vin_list["vehicle_id"].str.len() >= 11
    vin_list["vin_check"] = vin_list["vin_alnum_check"] & vin_list["vin_len_check"]
    
    # Strip the vins
    vin_list["vin_stripped"] = vin_list["vehicle_id"].str.strip()
    
    # Create 11-long vins
    vin_list["prepared_vins"] = vin_list["vehicle_id"].str[0:8]+"*"+vin_list["vehicle_id"].str[9:11]
    vin_list.loc[vin_list[vin_list["vin_check"]==False].index, "vin_corrected"] = np.NaN
    vin_list.loc[vin_list[vin_list["vin_check"]==True].index, "vin_corrected"] = vin_list.loc[vin_list[vin_list["vin_check"]==True].index, "prepared_vins"]
    
    # Clean up
    vin_list = vin_list.drop("prepared_vins", axis = 1)
    
    # Return
    return vin_list

In [618]:
def get_valid_zips(zip_list):
    # Get validity code
    zip_list["zip_valid_code"] = zip_list["zip"].apply(lambda x: check_valid_zip(x))
    
    # Get indices
    correct_zips_indices = zip_list[zip_list["zip_valid_code"]==1].index
    invalid_zips_indices = zip_list[zip_list["zip_valid_code"]==0].index
    two_part_zips_indices = zip_list[zip_list["zip_valid_code"]>1].index
    
    zip_list.loc[correct_zips_indices, "zip_corrected"] = zip_list.loc[correct_zips_indices, "zip"]
    zip_list.loc[invalid_zips_indices, "zip_corrected"] = np.NaN
    zip_list.loc[two_part_zips_indices, "zip_corrected"] = zip_list.loc[two_part_zips_indices, "zip"].str[0:5]
    
    return zip_list

In [600]:
def check_valid_zip(zip_code):
    zip_str = str(zip_code)
    split_zip = re.split("-", zip_str)
    if len(split_zip) == 2:
        if check_valid_zip(split_zip[0]) & check_valid_zip(split_zip[1]):
            return 2
        elif check_valid_zip(split_zip[0]):
            return 3
        else:
            return 0
    elif len(split_zip) == 1:
        # MUST ADDRESS STARTING "Os"
        # starting_o = re.match('^O', split_zip[0])
        matched = re.match("^\s*[0-9]*[0-9]{4}\.?0?\s*$", split_zip[0])
        if matched:
            return 1
        else:
            return 0
    else:
        return 0

In [ ]:
def process_chunk(chunk):
    checked_vins = chunk[["vehicle_id"]]
    

In [ ]:
for chunk in chunks:
    # Process the chunk
    
    # Add it to the dataframe
    
    # Save the dataframe

***